# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
#Your code here
import pandas as pd
#reading the file
df = pd.read_csv('homepage_actions.csv')
df

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
...,...,...,...,...
8183,2017-01-18 09:11:41.984113,192060,experiment,view
8184,2017-01-18 09:42:12.844575,755912,experiment,view
8185,2017-01-18 10:01:09.026482,458115,experiment,view
8186,2017-01-18 10:08:51.588469,505451,control,view


In [2]:
#No.of records
df.shape

(8188, 4)

INVESTIGATING

In [3]:
#checking our data info
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [4]:
#check for unique values in the column
df.action.unique()

array(['view', 'click'], dtype=object)

In [5]:
df.isna().sum()

timestamp    0
id           0
group        0
action       0
dtype: int64

In [19]:
df.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
8183    False
8184    False
8185    False
8186    False
8187    False
Length: 8188, dtype: bool

In [18]:
df['id'].duplicated().sum()

1860

In [9]:
df['group'].unique()

array(['experiment', 'control'], dtype=object)

In [8]:
df.describe(include="all")

,timestamp,id,group,action
count,8188,8188.000000,8188,8188
unique,8188,NaN,2,2
top,2016-09-24 17:42:27.839496,NaN,control,view
freq,1,NaN,4264,6328
mean,NaN,564699.749878,NaN,NaN
std,NaN,219085.845672,NaN,NaN
min,NaN,182988.000000,NaN,NaN
25%,NaN,373637.500000,NaN,NaN
50%,NaN,566840.500000,NaN,NaN
75%,NaN,758078.000000,NaN,NaN


In [22]:
#
experiment_data = df[df.group == 'experiment']
control_data = df[df.group == 'control']

In [23]:
len(experiment_data.id.unique())

2996

In [24]:
len(control_data.id.unique())

3332

Is there any overlap between the control and experiment groups

In [25]:
experiment_data.merge(control_data, how='inner', suffixes=('_click', '_view'), on='id')

,timestamp_click,id,group_click,action_click,timestamp_view,group_view,action_view


check to see if anyone clicked but didn't view

In [27]:
experiment_click_data = experiment_data[experiment_data.action == 'click']
experiment_view_data = experiment_data[experiment_data.action == 'view']
control_click_data = control_data[control_data.action == 'click']
control_view_data = control_data[control_data.action == 'view']

In [28]:
experiment_click_data.merge(experiment_view_data, on='id', suffixes=('_click', '_view'), how='left')

,timestamp_click,id,group_click,action_click,timestamp_view,group_view,action_view
0,2016-09-24 20:58:01.948663,349125,experiment,click,2016-09-24 20:57:20.336757,experiment,view
1,2016-09-24 21:06:27.553057,601714,experiment,click,2016-09-24 21:05:15.348935,experiment,view
2,2016-09-24 21:30:02.739756,487634,experiment,click,2016-09-24 21:29:19.766467,experiment,view
3,2016-09-24 23:01:12.108316,468601,experiment,click,2016-09-24 23:01:08.713402,experiment,view
4,2016-09-25 00:01:47.933853,555973,experiment,click,2016-09-25 00:00:47.700734,experiment,view
...,...,...,...,...,...,...,...
923,2017-01-17 09:27:37.808307,891635,experiment,click,2017-01-17 09:26:26.463667,experiment,view
924,2017-01-17 15:09:17.533736,865656,experiment,click,2017-01-17 15:08:41.151181,experiment,view
925,2017-01-17 20:14:12.483794,413359,experiment,click,2017-01-17 20:14:00.072907,experiment,view
926,2017-01-18 00:56:24.554729,344770,experiment,click,2017-01-18 00:55:55.026210,experiment,view


In [29]:
experiment_click_data.merge(experiment_view_data, on='id', suffixes=('_click', '_view'), how='left').info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 928 entries, 0 to 927
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   timestamp_click  928 non-null    object
 1   id               928 non-null    int64 
 2   group_click      928 non-null    object
 3   action_click     928 non-null    object
 4   timestamp_view   928 non-null    object
 5   group_view       928 non-null    object
 6   action_view      928 non-null    object
dtypes: int64(1), object(6)
memory usage: 58.0+ KB


In [30]:
experiment_click_members = 928

In [32]:
control_click_data.merge(control_view_data, on='id', suffixes=('_click', '_view'), how='left').info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 932 entries, 0 to 931
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   timestamp_click  932 non-null    object
 1   id               932 non-null    int64 
 2   group_click      932 non-null    object
 3   action_click     932 non-null    object
 4   timestamp_view   932 non-null    object
 5   group_view       932 non-null    object
 6   action_view      932 non-null    object
dtypes: int64(1), object(6)
memory usage: 58.2+ KB


In [33]:
control_click_members = 932

In [48]:
# experiment_view_members is the number of members of the experiment group
experimentviewers = len(experiment_data[experiment_data.action  == 'view'])
experimentviewers

2996

In [49]:
#control_view_members is the number of members of the control group
controlviewers = len(control_data[control_data.action == 'view'])
controlviewers

3332

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [50]:
#Your code here
import numpy as np
import statsmodels.api as sm


In [51]:
zscore, pval = sm.stats.proportions_ztest([control_click_members , experiment_click_members ], [controlviewers,experimentviewers ], alternative='smaller')
print(pval, zscore)

0.004415037788297902 -2.618563885349469


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [52]:
#Your code here

control_click_rate = control_click_members/controlviewers # calculated above
expected_experiment_clicks = control_click_rate * experimentviewers
expected_experiment_clicks

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [54]:
#Your code here
actual_expected_clicks = experiment_click_members
difference_in_clicks = actual_expected_clicks - expected_experiment_clicks
print(f'the difference in clicks was {difference_in_clicks}')
variance_control = experimentviewers * control_click_rate * (1 - control_click_rate)
standard_deviation = np.sqrt(variance_control)
print(f'the control standard deviation is {standard_deviation}')
standard_deviation_away = difference_in_clicks / standard_deviation
print(f'number of standard deviation away is {standard_deviation_away}')

the difference in clicks was 89.98319327731087
the control standard deviation is 24.568547907005815
number of standard deviation away is 3.6625360854823588


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [45]:
#Your code here
import scipy.stats as stats

p_value = stats.norm.sf(standard_deviation_away)
p_value

0.00012486528006951198

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **The p value is lower than the alpha of 0.05 thus we will reject the null hypothesis.**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.